<a href="https://colab.research.google.com/github/kheedogg/AIR/blob/main/%5BBI-000%5D%20TM%20TE%20CN%20start%20level%20distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://todomath.atlassian.net/browse/BI-163

**Request background and purpose**

"7세 이상 어린이 전용"에 대한 영향력 확인

**Details**

다음 기간의 토도수학/토도영어 신규 사용자 레벨 분포 데이터

- 2021.09.01-2021.10.30

- 2022.05.01-2022.06.15

In [ ]:
!pip install pyathena==2.5.4 > /dev/null
!pip install databricks-sql-connector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 196 kB 4.9 MB/s 
     |████████████████████████████████| 59 kB 4.6 MB/s 
  Created wheel for thrift: filename=thrift-0.16.0-cp37-cp37m-linux_x86_64.whl size=348309 sha256=f34e92ec062601f285441f390e4f6eb13b918e476e05accd134fdb04ca9b82d1
  Stored in directory: /root/.cache/pip/wheels/e6/d0/8c/11a1840ef59493f9221d13748a29482323f13f6da6b236c3de
Successfully built thrift


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import json

# download the aws file
key_file = drive.CreateFile({'id': '1Ioo4QebP5Kh1oEv2YbTej_MIpFA-SS_k'})
aws_access = json.loads(key_file.GetContentString())

# download the databrikcs file
key_file = drive.CreateFile({'id': '1MBD10AumpNtuqtzVDo9LJq4vj6898x0q'})
databricks_access = json.loads(key_file.GetContentString())

In [ ]:
# create athena connection
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

athena = connect(
    aws_access_key_id = aws_access['aws_access_key_id'],
    aws_secret_access_key = aws_access['aws_secret_access_key'],
    s3_staging_dir = 's3://enuma-dw/',
    region_name = 'us-west-2',
    cursor_class=PandasCursor).cursor()

In [ ]:
from databricks import sql
import pandas as pd

conn = sql.connect(server_hostname = databricks_access['server_hostname'],
                   http_path       = databricks_access['http_path'],
                   access_token    = databricks_access['access_token'])

In [ ]:
import pandas as pd

## 토도수학

In [ ]:
query_tm_start_level_prcnt = """
SELECT
  a.start_level
  , a.cnt AS cnt_before
  , b.cnt AS cnt_after
  , a.prcnt AS prcnt_before
  , b.prcnt AS prcent_after
FROM(
    SELECT
        UPPER(start_level) AS start_level
        , COUNT(DISTINCT userId) AS cnt
        , ROUND(100.0*COUNT(DISTINCT userId) / SUM(COUNT(DISTINCT userId)) Over(), 1) AS prcnt
    FROM(
        SELECT
            userId
            , CAST(user_regdate AS DATE) AS user_regdate
            , v_level AS start_level
            , ROW_NUMBER() OVER (PARTITION BY userId, CAST(user_regdate AS DATE) ORDER BY regdate DESC) AS row_num
        FROM bronze.todomath
        WHERE 1=1
            AND yyyymmdd BETWEEN '2022-05-01' AND '2022-06-15'
            AND env = 'live'
            AND device_localeCountry = 'CN'
            AND user_regdate BETWEEN DATE('2022-05-01') AND DATE('2022-06-15')
            AND v_event = 'tmLevelComplete'
            AND CAST(user_regdate AS DATE) = SUBSTR(regdate, 1, 10)
    )
    WHERE row_num = 1 
    GROUP BY 1
  ) a
  LEFT JOIN(
      SELECT
          UPPER(start_level) AS start_level
          , COUNT(DISTINCT userId) AS cnt
          , ROUND(100.0*COUNT(DISTINCT userId) / SUM(COUNT(DISTINCT userId)) Over(), 1) AS prcnt
      FROM(
          SELECT
              userId
              , CAST(user.user_regdate AS DATE) AS user_regdate
              , data.level AS start_level
              , ROW_NUMBER() OVER (PARTITION BY userId, CAST(user.user_regdate AS DATE) ORDER BY regdate DESC) AS row_num
          FROM bronze.todomath_parsed
          WHERE 1=1
              AND yyyymmdd BETWEEN '2022-05-01' AND '2022-06-15'
              AND env = 'live'
              AND device.localeCountry = 'CN'
              AND user.user_regdate BETWEEN DATE('2022-05-01') AND DATE('2022-06-15')
              AND data.event = 'tmLevelComplete'
              AND CAST(user.user_regdate AS DATE) = SUBSTR(regdate, 1, 10)
      )
      WHERE row_num = 1 
      GROUP BY 1
  ) b ON a.start_level = b.start_level
  ORDER BY a.start_level
"""

df = pd.read_sql(query_tm_start_level_prcnt, conn)
df

# 참고한 table in databricks
# befoer : bronze.todomath
# after : bronze.todomath_parsed

,start_level,cnt_before,cnt_after,prcnt_before,prcent_after
0,A,24333,22429,63.9,64.0
1,B,5561,5088,14.6,14.5
2,C,3831,3545,10.1,10.1
3,D,1198,1089,3.1,3.1
4,E,950,854,2.5,2.4
5,F,802,736,2.1,2.1
6,G,1422,1301,3.7,3.7
